In [1]:
URLS_PATH = r"C:\Users\User\Desktop\MultimodalRAGSystem\TheBatch\the_batch_urls.txt"

with open(URLS_PATH) as f:
    urls = f.readlines()
    urls = [url[:-2] for url in urls]

In [2]:
urls[:5]

['https://www.deeplearning.ai/the-batch',
 'https://www.deeplearning.ai/the-batch',
 'https://www.deeplearning.ai/the-batch/tag/letters',
 'https://www.deeplearning.ai/the-batch/tag/data-points',
 'https://www.deeplearning.ai/the-batch/tag/research']

In [3]:
from TheBatch.the_batch_data_loader import TheBatchDataLoader

loader = TheBatchDataLoader()

the_batch_loaded_data = []
for url in urls:
    the_batch_loaded_data.append(loader.load(url))

[2025-05-26 00:02:00,754 | python_pytorch_profiling_logger | INFO] -> ParserConfig validation.
[2025-05-26 00:02:00,755 | python_pytorch_profiling_logger | INFO] -> ParserConfig validation done successfully.
[2025-05-26 00:02:00,756 | python_pytorch_profiling_logger | INFO] -> TheBatchDataLoader fetching data from https://www.deeplearning.ai/the-batch using RequestsFetcher(timeout=10)
[2025-05-26 00:02:00,757 | python_pytorch_profiling_logger | INFO] -> RequestsFetcher fetching url https://www.deeplearning.ai/the-batch
[2025-05-26 00:02:01,953 | python_pytorch_profiling_logger | INFO] -> RequestsFetcher successfully fetched data from https://www.deeplearning.ai/the-batch
[2025-05-26 00:02:01,953 | python_pytorch_profiling_logger | INFO] -> TheBatchDataLoader successfully fetched data from https://www.deeplearning.ai/the-batch, now parsing using parser='html.parser'
[2025-05-26 00:02:01,954 | python_pytorch_profiling_logger | INFO] -> BS4Parser parsing data from https://www.deeplearning

In [4]:
the_batch_loaded_data[:5]

[ParsedData(url=https://www.deeplearning.ai/the-batch, parsed_tags=dict_keys(['article_title', 'publication_date', 'author_name', 'main_content', 'images', 'captions', 'tags', 'parahraph'])),
 ParsedData(url=https://www.deeplearning.ai/the-batch, parsed_tags=dict_keys(['article_title', 'publication_date', 'author_name', 'main_content', 'images', 'captions', 'tags', 'parahraph'])),
 ParsedData(url=https://www.deeplearning.ai/the-batch/tag/letters, parsed_tags=dict_keys(['article_title', 'publication_date', 'author_name', 'main_content', 'images', 'captions', 'tags', 'parahraph'])),
 ParsedData(url=https://www.deeplearning.ai/the-batch/tag/data-points, parsed_tags=dict_keys(['article_title', 'publication_date', 'author_name', 'main_content', 'images', 'captions', 'tags', 'parahraph'])),
 ParsedData(url=https://www.deeplearning.ai/the-batch/tag/research, parsed_tags=dict_keys(['article_title', 'publication_date', 'author_name', 'main_content', 'images', 'captions', 'tags', 'parahraph']))]

In [5]:
from TheBatch.the_batch_preprocessor import TheBatchPreprocessor

preprocessor = TheBatchPreprocessor()
preprocessor

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


TheBatchPreprocessor(text_extractor=SimpleBS4TextExtractor(separator=' ', strip=True, join_symbol='\n'), text_splitter=RecursiveTextSplitter(chunk_size=1500, chunk_overlap=0, separators=['\n\n', '\n', ' ', '']), image_loader=RequestsImageLoader(fetcher=RequestsFetcher(timeout=10)), image_describer=BLIPImageDescriber(pretrained_model_name_or_path='Salesforce/blip-image-captioning-base'))

In [24]:
docs = []
for sample in the_batch_loaded_data[-3:]:
    image_urls = [img['src'] for img in sample.images]
    docs.extend(preprocessor.preprocess(source_url=sample.url,
                                        elements=sample.get_all(),
                                        images_urls=image_urls)
                                        )
    

[2025-05-26 00:15:05,045 | python_pytorch_profiling_logger | INFO] -> TheBatchDataPreprocessor preprocessing, extracting text from elements using separator=' ' strip=True join_symbol='\n'
[2025-05-26 00:15:05,046 | python_pytorch_profiling_logger | INFO] -> SimpleBS4TextExtractor extracting text.
[2025-05-26 00:15:05,047 | python_pytorch_profiling_logger | INFO] -> SimpleBS4TextExtractor successfully extracted text.
[2025-05-26 00:15:05,048 | python_pytorch_profiling_logger | INFO] -> TheBatchDataPreprocessor successfully extracted text using separator=' ' strip=True join_symbol='\n', splitting extracted text using chunk_size=1500 chunk_overlap=0 separators=['\n\n', '\n', ' ', '']
[2025-05-26 00:15:05,048 | python_pytorch_profiling_logger | INFO] -> RecursiveTextSplitter splitting text from https://www.deeplearning.ai/the-batch/issue-287
[2025-05-26 00:15:05,062 | python_pytorch_profiling_logger | INFO] -> RecursiveTextSplitter successfully splitted text from https://www.deeplearning.a

In [36]:
import numpy as np
from Embedding.text_embedding import SentenceTransformerTextEmbedding

embedding_func = SentenceTransformerTextEmbedding()
docs_embeddings = np.vstack([embedding_func.encode([doc.content]) for doc in docs])

[2025-05-26 00:21:07,623 | python_pytorch_profiling_logger | INFO] -> SentenceTransformerTextEmbedding initialization.
[2025-05-26 00:21:07,625 | sentence_transformers.SentenceTransformer | INFO] -> Use pytorch device_name: cpu
[2025-05-26 00:21:07,626 | sentence_transformers.SentenceTransformer | INFO] -> Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
[2025-05-26 00:21:09,712 | python_pytorch_profiling_logger | INFO] -> SentenceTransformerTextEmbedding done successfully.
[2025-05-26 00:21:09,713 | python_pytorch_profiling_logger | INFO] -> SentenceTransformerTextEmbedding encoding sentences.
Batches: 100%|██████████| 1/1 [00:00<00:00, 70.58it/s]
[2025-05-26 00:21:09,731 | python_pytorch_profiling_logger | INFO] -> SentenceTransformerTextEmbedding successfuly encoded sentences.
[2025-05-26 00:21:09,732 | python_pytorch_profiling_logger | INFO] -> SentenceTransformerTextEmbedding encoding sentences.
Batches: 100%|██████████| 1/1 [00:00<00:00, 37.42it/s]
[202

In [43]:
from VectorStore.chroma_vector_store import ChromaVectorStore
from Internals.adapters import ChromaTextEmbeddingAdapter

adapted_embedding_func = ChromaTextEmbeddingAdapter(embedding_function=embedding_func)

vectorestore = ChromaVectorStore(embedding_function=adapted_embedding_func,
                                 collection_name="TheBatch")

In [44]:
vectorestore.add_documents(documents=docs,
                           embeddings=docs_embeddings)

[2025-05-26 00:23:48,786 | python_pytorch_profiling_logger | INFO] -> Adding new (documents, embeddings) to ChromaVectoreStore.
[2025-05-26 00:23:48,906 | python_pytorch_profiling_logger | INFO] -> New (documents, embeddings) successfully added to ChromaVectoreStore.


In [45]:
from langchain.prompts import PromptTemplate

News_article_prompt_template = PromptTemplate(
    input_variables=["context", "user_query"],
    template="""
You are an advanced AI assistant designed to provide comprehensive and well-reasoned answers to user queries based on relevant news articles. You have access to the following context, which may include text from news articles and associated images (described in text form).

Context:
{context}

Instructions:
- Carefully read the context above, which contains multiple news articles relevant to the user's question.
- If any images are described (e.g., captions, OCR, extracted text), incorporate them into your analysis.
- Summarize and synthesize the information to answer the user's question.
- Ensure your answer is accurate, relevant, and concise.

User query:
{user_query}

Your answer:
"""
)

In [46]:
from LLM.rag_llm import OllamaRAGLLM

model = OllamaRAGLLM(vectorstore=vectorestore,
                     prompt_template=News_article_prompt_template)

[2025-05-26 00:24:36,417 | python_pytorch_profiling_logger | INFO] -> OllamaRAGLLM initialization
[2025-05-26 00:24:36,418 | python_pytorch_profiling_logger | INFO] -> OllamaRAGLLM initialization done successfully.


In [50]:
query = "What is AI?"
answer = model.query(query)
answer

[2025-05-26 00:25:22,646 | python_pytorch_profiling_logger | INFO] -> OllamaRAGLLM processing user query: What is AI?
[2025-05-26 00:25:22,646 | python_pytorch_profiling_logger | INFO] -> Searching 5 similar documents for query: What is AI? in ChromaVectoreStore
[2025-05-26 00:25:22,647 | python_pytorch_profiling_logger | INFO] -> SentenceTransformerTextEmbedding encoding sentences.
Batches: 100%|██████████| 1/1 [00:00<00:00, 118.71it/s]
[2025-05-26 00:25:22,659 | python_pytorch_profiling_logger | INFO] -> SentenceTransformerTextEmbedding successfuly encoded sentences.
[2025-05-26 00:25:22,660 | python_pytorch_profiling_logger | INFO] -> 5 similar documents for query: What is AI? successfully retieved from ChromaVectoreStore
[2025-05-26 00:25:27,531 | python_pytorch_profiling_logger | INFO] -> OllamaRAGLLM successfully processed user query: What is AI?


RAGLLMResponse(user_query='What is AI?')

In [52]:
print(answer.llm_resopnse)

Based on the provided context, AI (Artificial Intelligence) refers to the use of computer systems to simulate human intelligence, including reasoning, learning, problem-solving, and perception. The articles mention various types of AI, such as:

1. General Artificial Intelligence (GenAI): Refers to AI that can perform any intellectual task a human can.
2. Narrow or Weak AI: Designed to perform specific tasks, like image recognition or language translation.
3. Human-Computer Interaction (HCI) AI: Systems that interact with users via speaking and listening, such as chatbots or virtual assistants.

The articles also touch on the applications of AI, including:

1. Autonomous vehicles
2. Natural Language Processing (NLP)
3. Computer vision
4. Robotics
5. Web scraping
6. Social media manipulation

Overall, AI encompasses a broad range of technologies and techniques that aim to create intelligent machines that can think and act like humans.

While the provided context doesn't provide an exhau

In [53]:
answer.relevant_docs

[TextDocument(id='8949146ce27d6accc2adb3c3722d8b919a6701ddc7e77cb0875624b6ad44284d-48aababa-73e2-459c-8307-fc8a099e9b75', content='can empower everyone to build with AI. Apr 23, 2025 OpenAI’s Five New Models, Hugging Face’s Open Robot, U.S. Tightens Grip on AI Chips, Text-Only LLMs Go Multimodal The Batch AI News and Insights: Even though I’m a much better Python than JavaScript developer, with AI assistance, I’ve been writing a lot of JavaScript code recently. Apr 16, 2025 Google Unveils Gemini 2.5, MCP Gains Momentum, Behind Sam Altman’s Fall and Rise, LLMs That Understand Misspellings The Batch AI News and Insights: I’ve noticed that many GenAI application projects put in automated evaluations (evals) of the system’s output probably later — and rely on humans to manually examine and judge outputs longer — than they should. Apr 09, 2025 Inside the Mind of Claude, Llama 4’s Mixture of Vision-Language Experts, More Open Multimodal Models, Neural Net for Tabular Data The Batch AI News a